## Test task
Implement a program that synchronizes two folders: source and
replica. The program should maintain a full, identical copy of source
folder at replica folder. Solve the test task by writing a program in
Python.

- Synchronization must be one-way: after the synchronization content of the
  replica folder should be modified to exactly match content of the source
  folder;
  Synchronization should be performed periodically;

- File creation/copying/removal operations should be logged to a file and to the
  console output;
  Folder paths, synchronization interval and log file path should be provided
  using the command line arguments;
  It is undesirable to use third-party libraries that implement folder
  synchronization;
  
- It is allowed (and recommended) to use external libraries implementing other
  well-known algorithms. For example, there is no point in implementing yet
  another function that calculates MD5 if you need it for the task – it is perfectly
  acceptable to use a third-party (or built-in) library.

Importing libraries and external functions

In [1]:
import os
import hashlib
import time
import json
import threading

Running test function

In [2]:
from simulate_fileops import *

Read config file

In [3]:
with open('../cfg/config.json') as f:
    config = json.load(f)
    
print(f'Default configuration')
config

Default configuration


{'source': '../raw/source_folder',
 'replica': '../replica_folder',
 'log': '../.log',
 'log_file': '/sync_log.txt',
 'sync_interval': 60}

In [4]:
source_folder = config['source']
replica_folder = config['replica']
log_folder = config['log']
log_file_path = log_folder + config['log_file']
sync_interval = config['sync_interval'] # this is in seconds

Synchronization of two folders

In [5]:
def calculate_md5(file_path: str) -> hashlib.md5:
    """Calculate the MD5 hash of a file

    Args:
        file_path (str): file path

    Returns:
        _type_: hash_md5
    """
    
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def copy_file(source_file: str, replica_file: str):
    """Copy file from source to replica

    Args:
        source_file (str): source file path
        replica_file (str): replaica file path
    """
    
    with open(source_file, 'rb') as src_file:
        with open(replica_file, 'wb') as dest_file:
            for chunk in iter(lambda: src_file.read(4096), b""):
                dest_file.write(chunk)

def create_folders():
    """Create folders if they don't exist
    """
    if not os.path.exists(source_folder):
        os.makedirs(source_folder)

    if not os.path.exists(replica_folder):
        os.makedirs(replica_folder)

    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

In [6]:
def synchronize_folders(run_sync: bool = True):
    """Sync folders and log each addition, modification and removal of files
    """
    
    # Create source folder, replica and log folders if they doesn't exist
    create_folders()
    
    # Tracking replica folder
    replica_state = {}
    
    current_time = time.ctime()
    
    print(f"[{current_time}] Syncing {source_folder} and {replica_folder} folders")
    
    print(f"[{current_time}] Sync interval: {sync_interval} seconds")
    
    print(f"[{current_time}] Logging to {log_file_path}")
    
    with open(log_file_path, "w") as log_file:
        log_file.write(f"[{current_time}] Syncing {source_folder} and {replica_folder} folders\n")
        log_file.write(f"[{current_time}] Sync interval: {sync_interval} seconds\n")
    
    while run_sync:
        for root, dirs, files in os.walk(source_folder):
            for file_name in files:
                source_file_path = os.path.join(root, file_name).replace('\\', '/')
                replica_file_path = os.path.join(replica_folder, os.path.relpath(source_file_path, source_folder)).replace('\\', '/')

                # MD5 hash of source file
                source_file_md5 = calculate_md5(source_file_path)

                # Verify here
                if replica_file_path in replica_state:
                    replica_file_md5 = replica_state[replica_file_path]
                else:
                    replica_file_md5 = ""

                # checkinfg if file is new or has been modified
                if source_file_md5 != replica_file_md5:
                    copy_file(source_file_path, replica_file_path)
                    replica_state[replica_file_path] = source_file_md5
                    current_time = time.ctime()
                    print(f"[{current_time}] Copied: {source_file_path} -> {replica_file_path}")
                    with open(log_file_path, "a") as log_file:
                        log_file.write(f"[{current_time}] Copied: {source_file_path} -> {replica_file_path}\n")

        # Check for files to delete in replica folder
        for replica_file_path, replica_file_md5 in list(replica_state.items()):
            source_file_path = os.path.join(source_folder, os.path.relpath(replica_file_path, replica_folder))
            # deletes in replica folder if it was deleted in source
            if not os.path.exists(source_file_path):
                os.remove(replica_file_path)
                del replica_state[replica_file_path]
                current_time = time.ctime()
                print(f"[{current_time}] Deleted: {replica_file_path}")
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"[{time.ctime()}] Deleted: {replica_file_path}\n")

        time.sleep(sync_interval)

### Threading

In [7]:
try:

    # Run the synchronization in a separate thread
    sync_thread = threading.Thread(target=synchronize_folders, daemon=False)

    # Simulate file operations in a separate thread
    sim_fileops_thread = threading.Thread(target=simulate_file_operations, daemon=False)

    # Start threads
    sync_thread.start()
    sim_fileops_thread.start()

    # Stops the folders synchronization
    input(f"Press Enter to finish folder synchronization!")
    run_sync = False

    # Logging the end of the synchronization application
    current_time = time.ctime()
    print(f"[{current_time}] Stopping synchronization thread...")
    with open(log_file_path, "a") as log_file: # append to log file
        log_file.write(f"[{current_time}] Stopping folders synchronization.\n")

except Exception as e:
    current_time = time.ctime()
    print(f"{[current_time]} Error: {e}")
    with open(log_file_path, "a") as log_file: # append exception to log file
        log_file.write(f"{[current_time]} Error: {e}\n")


[Thu Nov  9 08:46:23 2023] File operations started!
[Thu Nov  9 08:46:23 2023] Syncing ../raw/source_folder and ../replica_folder folders
[Thu Nov  9 08:46:23 2023] Sync interval: 60 seconds
[Thu Nov  9 08:46:23 2023] Logging to ../.log/sync_log.txt
[Thu Nov  9 08:48:23 2023] Copied: ../raw/source_folder/dummy_text.txt -> ../replica_folder/dummy_text.txt
[Thu Nov  9 08:49:23 2023] Copied: ../raw/source_folder/dummy_text.txt -> ../replica_folder/dummy_text.txt
[Thu Nov  9 08:49:23 2023] Copied: ../raw/source_folder/ratings_matrix.csv -> ../replica_folder/ratings_matrix.csv
[Thu Nov  9 08:50:23 2023] Deleted: ../replica_folder/ratings_matrix.csv
[Thu Nov  9 08:51:23 2023] Copied: ../raw/source_folder/ratings_matrix.csv -> ../replica_folder/ratings_matrix.csv
[Thu Nov  9 08:52:23 2023] File operations finished!
[Thu Nov  9 08:52:23 2023] Copied: ../raw/source_folder/Veeam-Logo.png -> ../replica_folder/Veeam-Logo.png
[Thu Nov  9 08:56:36 2023] Stopping synchronization thread...


## Standalone version

In [ ]:
import os
import time
import hashlib
import argparse

def calculate_md5(file_path: str) -> hashlib.md5:
    """Calculate the MD5 hash of a file

    Args:
        file_path (str): file path

    Returns:
        _type_: hash_md5
    """
    
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def copy_file(source_file: str, replica_file: str):
    """Copy file from source to replica

    Args:
        source_file (str): source file path
        replica_file (str): replaica file path
    """
    
    with open(source_file, 'rb') as src_file:
        with open(replica_file, 'wb') as dest_file:
            for chunk in iter(lambda: src_file.read(4096), b""):
                dest_file.write(chunk)

def create_folders(source_folder: str, replica_folder: str, log_folder: str):
    """Create folders if they don't exist
    """
    if not os.path.exists(source_folder):
        os.makedirs(source_folder)

    if not os.path.exists(replica_folder):
        os.makedirs(replica_folder)

    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

def synchronize_folders(source_folder: str, replica_folder: str, log_file_path: str, sync_interval: int, run_sync: bool = True):
    """Sync folders and log each addition, modification and removal of files
    """
    
    # Tracking replica folder
    replica_state = {}
    
    current_time = time.ctime()
    
    print(f"[{current_time}] Syncing {source_folder} and {replica_folder} folders")
    
    print(f"[{current_time}] Sync interval: {sync_interval} seconds")
    
    print(f"[{current_time}] Logging to {log_file_path}")
    
    with open(log_file_path, "w") as log_file:
        log_file.write(f"[{current_time}] Syncing {source_folder} and {replica_folder} folders\n")
        log_file.write(f"[{current_time}] Sync interval: {sync_interval} seconds\n")
    
    while run_sync:
        for root, dirs, files in os.walk(source_folder):
            for file_name in files:
                source_file_path = os.path.join(root, file_name).replace('\\', '/')
                replica_file_path = os.path.join(replica_folder, os.path.relpath(source_file_path, source_folder)).replace('\\', '/')

                # MD5 hash of source file
                source_file_md5 = calculate_md5(source_file_path)

                # Verify here
                if replica_file_path in replica_state:
                    replica_file_md5 = replica_state[replica_file_path]
                else:
                    replica_file_md5 = ""

                # checkinfg if file is new or has been modified
                if source_file_md5 != replica_file_md5:
                    copy_file(source_file_path, replica_file_path)
                    replica_state[replica_file_path] = source_file_md5
                    current_time = time.ctime()
                    print(f"[{current_time}] Copied: {source_file_path} -> {replica_file_path}")
                    with open(log_file_path, "a") as log_file:
                        log_file.write(f"[{current_time}] Copied: {source_file_path} -> {replica_file_path}\n")

        # Check for files to delete in replica folder
        for replica_file_path, replica_file_md5 in list(replica_state.items()):
            source_file_path = os.path.join(source_folder, os.path.relpath(replica_file_path, replica_folder))
            # deletes in replica folder if it was deleted in source
            if not os.path.exists(source_file_path):
                os.remove(replica_file_path)
                del replica_state[replica_file_path]
                current_time = time.ctime()
                print(f"[{current_time}] Deleted: {replica_file_path}")
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"[{time.ctime()}] Deleted: {replica_file_path}\n")

        time.sleep(sync_interval)

def main():
    
    # Parse arguments: source, replica, log and sync_interval
    parser = argparse.ArgumentParser(description='Sync folders')
    parser.add_argument('--source', type=str, help='Source folder')
    parser.add_argument('--replica', type=str, help='Replica folder')
    parser.add_argument('--log', type=str, help='Log folder')
    parser.add_argument('--interval', type=int, help='Sync interval in seconds')
    args = parser.parse_args()

    if args.source:
        source_folder = args.source

    if args.replica:
        replica_folder = args.replica

    if args.log:
        log_folder = args.log

    if args.interval:
        sync_interval = args.interval

    # Create source folder, replica and log folders if they doesn't exist
    create_folders(source_folder, replica_folder, log_folder)
    
    try:

        log_file_path = log_folder + f'/sync_log_{time.ctime()}.txt'.replace(' ', '_')

        # Synchronization
        synchronize_folders(source_folder, replica_folder, log_file_path, sync_interval)

        # Logging the end of the synchronization application
        current_time = time.ctime()
        print(f"[{current_time}] Stopping synchronization thread...")
        with open(log_file_path, "a") as log_file: # append to log file
            log_file.write(f"[{current_time}] Stopping folders synchronization.\n")

    except Exception as e:
        # log exception if not KeyboardInterrupt
        if not isinstance(e, KeyboardInterrupt):
            current_time = time.ctime()
            print(f"{[current_time]} Error: {e}")
            with open(log_file_path, "a") as log_file: # append exception to log file
                log_file.write(f"{[current_time]} Error: {e}\n")

if __name__ == '__main__':
    main()
    
main()

## Running tests

- **SETUP:**
    - ``sync_app.py`` is run with the following params:
        - ``source_folder``: [``../raw/source_folder``](../raw/source_folder)
        - ``replica_folder``: [``../replica_folder``](../replica_folder)
        - ``log_file``: [``../.log/sync_log.txt``](../.log/sync_log.txt)
        - ``sync_interval``: 60 seconds

- **TESTS:**
    - File addition, modification and removal will be tested.
    - Blank file ``dummy_text.txt`` is copied to source_folder
    - Dummy file ``dummy_text.txt`` is modified in ``source_folder`` by copying the content from ``dummy_text.txt`` in [``../raw/example_files/dummy_text-modify_1.txt``](../raw/example_files/dummy_text-modify_1.txt)
    - This process will be repeated again with [``../raw/example_files/dummy_text-modify_2.txt``](../raw/example_files/dummy_text-modify_2.txt).
    - The image file [``../raw/example_files/Veeam-Logo.png``](../raw/example_files/Veeam-Logo.png) will be copied to ``source_folder``.
    - The CSV file [``../raw/example_files/ratings_matrix.csv``](../raw/example_files/ratings_matrix.csv) will be copied to ``source_folder``.

- **EXPECTED RESULTS:**
    - It is expected that the ``dummy_text.txt`` file will be copied to [``../replica_folder``](../replica_folder) and removed from it if it no longer exists inside the ``source_folder``.

    - Changes made to the ``dummy_text.txt`` file in ``source_folder`` will be reflected in the ``dummy_text.txt`` file in ``replica_folder``.

    - It is expect that the log file will register the operation at [``../.log/sync_log.txt``](../.log/sync_log.txt).

### Code created to simulate file operations in [``./simulate_fileops.py``](./simulate_fileops.py)

````python
import shutil
import time
import os

def simulate_file_operations():
    """Simulate file operations by copying files to the source folder and removing them after a certain time."""
    
    source_folder = '../raw/source_folder'
    example_files = '../raw/example_files'

    # Sleep for 80 seconds
    time.sleep(80)

    # Copy dummy_text.txt
    shutil.copy(os.path.join(example_files, 'dummy_text.txt'), source_folder)

    # Sleep for 70 seconds
    time.sleep(70)

    # Copy modified dummy_text.txt
    shutil.copy(os.path.join(example_files, 'dummy_text-modify_1.txt'), os.path.join(source_folder, 'dummy_text.txt'))

    # Sleep for 15 seconds
    time.sleep(15)

    # Copy modified dummy_text.txt again
    shutil.copy(os.path.join(example_files, 'dummy_text-modify_2.txt'), os.path.join(source_folder, 'dummy_text.txt'))

    # Sleep for 5 seconds
    time.sleep(5)

    # Copy ratings_matrix.csv
    shutil.copy(os.path.join(example_files, 'ratings_matrix.csv'), source_folder)

    # Sleep for 90 seconds
    time.sleep(90)

    # Copy ratings_matrix.csv again
    shutil.copy(os.path.join(example_files, 'ratings_matrix.csv'), source_folder)

    # Sleep for 20 seconds
    time.sleep(20)

    # Copy Veeam-Logo.png
    shutil.copy(os.path.join(example_files, 'Veeam-Logo.png'), source_folder)

    # Sleep for 10 seconds
    time.sleep(10)

    # Remove Veeam-Logo.png
    os.remove(os.path.join(source_folder, 'Veeam-Logo.png'))

    # Sleep for 70 seconds
    time.sleep(70)

    # Copy Veeam-Logo.png again
    shutil.copy(os.path.join(example_files, 'Veeam-Logo.png'), source_folder)

if __name__ == "__main":
    simulate_file_operations()



## What could be improved:

- Logging could be improved by using a logger object and be appropriatly handled by it since python has a built-in library for that.

- The code could be improved by using a class to handle the syncronization process.

- Testing: While I have tested the code enough to check that it works as expected, I haven't used any testing library to do so due to time constraints.

- A GUI could be implemented to make it more user friendly.

## Comments:

- While as implementing I have noticed ways to optimize the code, I have decided to leave it as it is to show my thought process and how I have improved the code as I was implementing it.

- This version should be considered as **working as required**, but not as a **final version** which, given more time, I could put more thought in it.

- I have decided to use the ``os`` library to handle the file system operations which have most of the time been used in other projects I have done before and were enough for the job, but I have noticed that there is a library called ``shutil`` which could be used to handle the file system operations in a more efficient way. I have decided to not use it because a version **from scratch** could help me deliver a version as fast as possible due to more familiarity.

- I haven't used any third-party library to handle the MD5 hash, but I have used the built-in library ``hashlib`` to do so.

- I have chosen the Jupyter Notebook to write the code because it is a tool I am familiar which makes it easeir for me to use, showcase and illustrate my thought process.

- I am aware that I could make everything using just linux commands like ``rsync`` and automate the process using cron, but I am more confortable with python code despite being familiar with many unix commands.

## References:

I have used the following references to help me implement the code:

- Other projects made by myself where I needed to handle file system operations and multithreading.

- [VERIFY MD5 / SHA256 Hash or Checksum on Linux - File Security (Ubuntu)](https://www.youtube.com/watch?v=uIIn6qVGOJQ) to review some concepts about MD5 hash and test it locally in my system.

- [How To Detect File Changes with Python (and send notification)](https://www.youtube.com/watch?v=lVDajXJEpmg) - is totally different from what I have implemented but it helped me as a first concept.

- [Python Documentation](https://docs.python.org/3/library/os.html) to check os documentation.

- [Python Documentation](https://docs.python.org/3/library/hashlib.html) to check hashlib documentation.


